In [ ]:
!pip install lifelines xgboost==2.0.3 pycox torchtuples
!pip install scikit-learn==1.2.2
!pip install scikit-survival==0.19.0

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier, plot_importance
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os
import json
import numpy as np
import pandas as pd
from sklearn.utils import resample
from catboost import CatBoostClassifier

In [16]:
df = pd.read_csv("/kaggle/input/football-final-dataset/5k_pos_new_v2.csv")
df = df.replace({-1:np.nan})

# Define the feature groups
team_fixture_cols = [
    'prev_inj_team_fixture_1', 'prev_inj_team_fixture_2',
    'prev_inj_team_fixture_3', 'prev_inj_team_fixture_4',
    'prev_inj_team_fixture_5'
]

opp_fixture_cols = [
    'prev_opp_team_fixture_1', 'prev_opp_team_fixture_2',
    'prev_opp_team_fixture_3', 'prev_opp_team_fixture_4',
    'prev_opp_team_fixture_5'
]


df = df[~(df[team_fixture_cols].isna().all(axis=1) | df[opp_fixture_cols].isna().all(axis=1))].reset_index(drop=True)
df.to_csv("new_800_pos.csv",index=False)

In [17]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import pandas as pd
import numpy as np
df = pd.read_csv("/kaggle/input/football-final-dataset/1k_pos_sample.csv")
df2 = pd.read_csv("/kaggle/input/football-final-dataset/2k_neg_sample.csv")
df3 = pd.read_csv("/kaggle/input/football-final-dataset/new_100_pos_samples.csv")
df4 = pd.read_csv("new_800_pos.csv")

df = pd.concat([df,df2,df3,df4]).drop("Unnamed: 0",axis=1).reset_index(drop=True)

df

,player_id,fixture_id,date,team_id,league_id,injury_season,reason,injuried,opponent_team_id,prev_inj_team_fixture_1,...,team_vs_opp_offsides_diff_5,team_vs_opp_poss_diff_5,team_vs_opp_pass_acc_diff_5,team_vs_opp_shots_ratio_5,team_vs_opp_sog_ratio_5,team_vs_opp_fouls_ratio_5,team_vs_opp_corners_ratio_5,team_vs_opp_offsides_ratio_5,team_vs_opp_poss_ratio_5,team_vs_opp_pass_acc_ratio_5
0,1557,1208578,2024-10-19 16:30:00+00:00,547,140,2024,Injury,1,548.0,1208570.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1557,1299023,2024-10-22 19:00:00+00:00,547,2,2024,Muscle Injury,1,656.0,1208578.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1557,1208589,2024-10-26 16:30:00+00:00,547,140,2024,Muscle Injury,1,534.0,1299023.0,...,0.0,11.000000,0.083478,1.363636,1.166667,1.583333,3.50,NaN,1.207547,1.102213
3,1557,1208596,2024-11-02 15:15:00+00:00,547,140,2024,Muscle Injury,1,537.0,1311446.0,...,1.0,25.500000,0.127389,4.142857,3.250000,2.066667,3.25,1.50,1.653846,1.167544
4,1561,1208527,2024-09-01 19:30:00+00:00,543,140,2024,Muscle Injury,1,541.0,1287591.0,...,-3.0,-3.500000,-0.054842,2.235294,1.777778,2.600000,2.20,0.25,0.945312,0.940172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4389,30924,882104,2023-05-03 19:00:00+00:00,497,135,2022,Knee Injury,1,1579.0,882098.0,...,0.0,-2.666667,-0.010171,0.607143,0.272727,0.157895,0.50,1.00,0.944056,0.987539
4390,30924,1022972,2023-05-11 19:00:00+00:00,497,3,2022,Knee Injury,1,168.0,882116.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4391,30974,946887,2022-10-04 19:00:00+00:00,505,2,2022,Injury,1,529.0,881853.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4392,30974,881867,2022-10-08 13:00:00+00:00,505,135,2022,Injury,1,488.0,946887.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import pandas as pd
import numpy as np

def add_temporal_injury_features(df):
    """
    Extend `df` with rich temporal injury features.
    Requires:
      - df["player_id"]
      - df["date"] (injury datetime)
      - df["injury_season"]
      - prev_inj_team_fixture_1..5
    """
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    
    # Base sorted subset
    inj = df[['player_id','date','injury_season']].sort_values(['player_id','date'])
    
    # First & previous dates
    inj['first_date'] = inj.groupby('player_id')['date'].transform('min')
    inj['prev_date']  = inj.groupby('player_id')['date'].shift(1)
    inj['days_between_prev_injury'] = (inj['date'] - inj['prev_date']).dt.days
    
    # Merge back
    df = df.merge(
        inj[['player_id','date','first_date','prev_date','days_between_prev_injury']],
        on=['player_id','date'], how='left'
    )
    df['days_since_first_injury'] = (df['date'] - df['first_date']).dt.days
    
    # Counts per season / total
    df['inj_count_season'] = df.groupby(['player_id','injury_season'])['date'].transform('count')
    df['inj_count_total']  = df.groupby('player_id')['date'].transform('count')
    
    # Rolling counts (1,3,6,12 months)
    for m in [1,3,12]:
        col = f'inj_count_last_{m}m'
        df[col] = df.apply(
            lambda r: df[
                (df['player_id']==r['player_id']) &
                (df['date'] >= r['date'] - pd.DateOffset(months=m)) &
                (df['date'] <  r['date'])
            ].shape[0],
            axis=1
        )
    
    # Fraction in last 6m
    df['inj_frac_last_6m'] = df['inj_count_last_6m'] / df['inj_count_total']
    
    # Prev fixtures count & rates
    prev_cols = [f'prev_inj_team_fixture_{i}' for i in range(1,6)]
    df['prev_fixtures_count'] = df[prev_cols].notna().sum(axis=1)
    df['inj_rate_per_fixture_6m'] = (
        df['inj_count_last_6m'] / df['prev_fixtures_count'].replace(0,np.nan)
    )
    
    # Sequence & gap statistics
    df['inj_seq_overall'] = df.groupby('player_id').cumcount() + 1
    df['inj_seq_season']  = df.groupby(['player_id','injury_season']).cumcount() + 1
    df['avg_days_between_prev'] = df.groupby('player_id')['days_between_prev_injury'].transform('mean')
    df['std_days_between_prev'] = df.groupby('player_id')['days_between_prev_injury'].transform('std')
    
    # Previous season count
    df['inj_count_prev_season'] = (
        df.groupby('player_id')['inj_count_season'].shift(1)
          .fillna(0)
          .astype(int)
    )
    
    # Rates per time & per fixture total
    df['inj_rate_per_year'] = df['inj_count_total'] / (
        (df['days_since_first_injury'] / 365).replace(0,np.nan)
    )
    df['inj_rate_per_year'] = df['inj_rate_per_year'].fillna(df['inj_count_total'])
    
    df['inj_rate_per_fixture_total'] = (
        df['inj_count_total'] / df['prev_fixtures_count'].replace(0,np.nan)
    )
    
    # Calendar / seasonality
    df['inj_on_weekend']   = (df['date'].dt.weekday >= 5).astype(int)
    df['injury_month']     = df['date'].dt.month
    df['injury_dow']       = df['date'].dt.weekday
    df['inj_doy']          = df['date'].dt.dayofyear
    df['inj_weekofyear']   = df['date'].dt.isocalendar().week
    df['inj_quarter']      = df['date'].dt.quarter
    
    # Clean up
    df = df.drop(columns=['first_date','prev_date'], errors='ignore')
    return df

# Example usage:
df = add_temporal_injury_features(df)
df.head()


,player_id,fixture_id,date,team_id,league_id,injury_season,reason,injuried,opponent_team_id,prev_inj_team_fixture_1,...,std_days_between_prev,inj_count_prev_season,inj_rate_per_year,inj_rate_per_fixture_total,inj_on_weekend,injury_month,injury_dow,inj_doy,inj_weekofyear,inj_quarter
0,1557,1208578,2024-10-19 16:30:00+00:00,547,140,2024,Injury,1,548.0,1208570.0,...,1.732051,0,4.000000,0.8,1,10,5,293,42,4
1,1557,1299023,2024-10-22 19:00:00+00:00,547,2,2024,Muscle Injury,1,656.0,1208578.0,...,1.732051,4,486.666667,0.8,0,10,1,296,43,4
2,1557,1208589,2024-10-26 16:30:00+00:00,547,140,2024,Muscle Injury,1,534.0,1299023.0,...,1.732051,4,208.571429,0.8,1,10,5,300,43,4
3,1557,1208596,2024-11-02 15:15:00+00:00,547,140,2024,Muscle Injury,1,537.0,1311446.0,...,1.732051,4,112.307692,0.8,1,11,5,307,44,4
4,1561,1208527,2024-09-01 19:30:00+00:00,543,140,2024,Muscle Injury,1,541.0,1287591.0,...,3.872983,0,5.000000,1.0,1,9,6,245,35,3


In [19]:
df[['fixture_id',
        'injury_season','opponent_team_id', 'prev_inj_team_fixture_1',
       'prev_inj_team_fixture_2', 'prev_inj_team_fixture_3',
       'prev_inj_team_fixture_4', 'prev_inj_team_fixture_5',
       'prev_opp_team_fixture_1', 'prev_opp_team_fixture_2',
       'prev_opp_team_fixture_3', 'prev_opp_team_fixture_4',
       'prev_opp_team_fixture_5']] = df[[ 'fixture_id',
       'injury_season','opponent_team_id', 'prev_inj_team_fixture_1',
       'prev_inj_team_fixture_2', 'prev_inj_team_fixture_3',
       'prev_inj_team_fixture_4', 'prev_inj_team_fixture_5',
       'prev_opp_team_fixture_1', 'prev_opp_team_fixture_2',
       'prev_opp_team_fixture_3', 'prev_opp_team_fixture_4',
       'prev_opp_team_fixture_5']].fillna(-1).astype(int)

In [20]:
df.to_csv("final_df2.csv",index=False)

In [10]:
"""
expand_final_df2.py  –  from wide match‑level data → minute‑level long table
---------------------------------------------------------------------------
* input : final_df2.csv  (your Stage‑2 output with one row per injury event)
* output: final_df2_long_by_minute.parquet
          (one row per player × minute, randomised minute‑stats)

Requirements
------------
pip install pandas tqdm requests pyarrow  # pyarrow for Parquet
Set the environment variable API_FOOTBALL_KEY with your API key.

Usage
-----
python expand_final_df2.py
"""
import os, json, time, math, random, requests
import numpy as np
import pandas as pd
from tqdm import tqdm

# ------------------------------------------------------------------ config --
API_KEY   = "xxx"
BASE_URL  = "https://v3.football.api-sports.io"
HEADERS   = {"x-rapidapi-host": "v3.football.api-sports.io",
             "x-rapidapi-key":  API_KEY}
STATS_DIR = "stats_cache"
FULL_TIME = 95            # 90 + typical stoppage
TAIL_BIAS = 1.3           # >1.0 → more events in minutes 76‑95
os.makedirs(STATS_DIR, exist_ok=True)

# ----------------------------------------------------------- helper funcs --
def get_fixture_stats(fixture_id: int) -> dict:
    """
    Returns a dict { team_id : {metric: value, …}, … } for the fixture.
    Caches JSON to disk so subsequent runs are instant.
    """
    cache_f = f"{STATS_DIR}/{fixture_id}.json"
    if os.path.exists(cache_f):
        with open(cache_f) as fh:
            return json.load(fh)

    for attempt in range(5):
        try:
            r = requests.get(f"{BASE_URL}/fixtures/statistics?fixture={fixture_id}",
                             headers=HEADERS, timeout=10)
            if r.status_code == 429:     # rate‑limit → exponential back‑off
                time.sleep(1.5 * (attempt + 1))
                continue
            data = r.json().get("response", [])
            team_stats = {}
            for block in data:          # home & away
                team_id = str(block["team"]["id"])
                stats = {s["type"].lower().replace(" ", "_"): s["value"]
                         for s in block["statistics"]}
                team_stats[team_id] = stats
            with open(cache_f, "w") as fh:
                json.dump(team_stats, fh)
            return team_stats
        except Exception as e:
            print(f"!! error fetching fixture {fixture_id}: {e}")
            time.sleep(2)
    return {}

import math, numpy as np

def random_cumulative(total, full_time=95, bias_tail=1.3):
    """
    Robustly split 'total' into 'full_time' biased bins and return cumulative list.
    Any NaN, None, or negative input is treated as missing → list of NaNs.
    """
    # --- validate total ---
    if total is None:
        return [math.nan] * full_time
    try:
        total_val = int(total)
    except (ValueError, TypeError):
        return [math.nan] * full_time          # non‑numeric like "‑" or "55%"
    if total_val < 0:
        return [math.nan] * full_time          # guard against n < 0
    if total_val == 0:
        return [0] * full_time

    # --- biased multinomial split ---
    probs = np.ones(full_time)
    probs[full_time - 20 :] *= bias_tail       # heavier tail
    probs /= probs.sum()

    per_min = np.random.multinomial(total_val, probs)
    return np.cumsum(per_min).tolist()


# -------------------------------------------------------------- pipeline --
def main():
    print("🔹 loading final_df2.csv …")
    df = pd.read_csv("final_df2.csv")

    # step 1 – fetch stats per fixture (one API call each, cached)
    print("🔹 downloading statistics once per fixture …")
    stats_map = {fid: get_fixture_stats(fid)
                 for fid in tqdm(df.fixture_id.unique())}

    # step 2 – fill missing injury minute with reasonable random
    if "injury_elapsed_min" not in df.columns:
        df["injury_elapsed_min"] = np.nan
    
    df["injury_elapsed_min"] = df["injury_elapsed_min"].apply(
        lambda x: x if pd.notna(x) else random.randint(10, 85)
    )

    # step 3 – build long table (player × minute)
    print("🔹 expanding rows minute‑by‑minute …")
    long_rows = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        fid, tid, pid = row.fixture_id, str(row.team_id), row.player_id
        totals = stats_map.get(fid, {}).get(tid, {})

        # pre‑sample cumulative series for each metric once
        cum_series = {k: random_cumulative(v) for k, v in totals.items()
                      if isinstance(v, (int, float))}

        for minute in range(1, FULL_TIME + 1):
            minute_stats = {f"{k}_cum": cum_series[k][minute-1]
                            for k in cum_series}
            long_rows.append({
                "player_id": pid,
                "fixture_id": fid,
                "team_id": tid,
                "minute": minute,
                "injuried": row.injuried,
                "injury_elapsed_min": row.injury_elapsed_min,
                **minute_stats
            })

    long_df = pd.DataFrame(long_rows)
    long_df.to_parquet("final_df2_long_by_minute.parquet", index=False)
    print("✅  saved → final_df2_long_by_minute.parquet")
    print(long_df.head())

if __name__ == "__main__":
    main()


🔹 loading final_df2.csv …
🔹 downloading statistics once per fixture …



100%|██████████| 21/21 [00:00<00:00, 7627.33it/s]


🔹 expanding rows minute‑by‑minute …



100%|██████████| 21/21 [00:00<00:00, 420.66it/s]

✅  saved → final_df2_long_by_minute.parquet
   player_id  fixture_id team_id  minute  injuried  injury_elapsed_min  \
0       1557     1208578     547       1         1                  43   
1       1557     1208578     547       2         1                  43   
2       1557     1208578     547       3         1                  43   
3       1557     1208578     547       4         1                  43   
4       1557     1208578     547       5         1                  43   

   shots_on_goal_cum  shots_off_goal_cum  total_shots_cum  blocked_shots_cum  \
0                  0                   0                0                  0   
1                  0                   0                0                  0   
2                  0                   0                0                  0   
3                  0                   0                0                  0   
4                  0                   0                0                  0   

   ...  shots_outsidebox_cum  

In [36]:
import pandas as pd
import numpy as np
import joblib
import warnings

from lifelines import NelsonAalenFitter
from sksurv.metrics import concordance_index_ipcw
from sksurv.util    import Surv
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing  import StandardScaler
from sklearn.pipeline       import Pipeline
from xgboost                import XGBRegressor

warnings.filterwarnings("ignore")

def main():
    # ——— load data ——————————————————————————————————————————————
    df = pd.read_parquet("/kaggle/input/football-models/final_df2_long_by_minute.parquet").dropna()
    
    # 1) raw T / E
    T = df["minute"].astype(int)
    E = (df["injuried"].astype(bool) & (T == df["injury_elapsed_min"])).values
    
    # 2) Nelson–Aalen transform H(T)
    naf = NelsonAalenFitter()
    naf.fit(T, event_observed=E)
    H = pd.Series(naf.cumulative_hazard_at_times(T).values, index=df.index)
    
    # 3) survival-structured array (for concordance eval only)
    y_struct = Surv.from_arrays(E, T)
    
    # 4) feature matrix
    X = ( df
          .drop(columns=["injuried","injury_elapsed_min","minute",
                         "player_id","fixture_id","team_id"])
          .select_dtypes(include=[np.number])
          .fillna(0)
        )
    
    groups = df["player_id"].values
    strata = df["injuried"].astype(int).values
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    results = {"XGB_NA": []}
    
    for fold, (tr, va) in enumerate(cv.split(X, strata, groups), 1):
        X_tr, X_va = X.iloc[tr],    X.iloc[va]
        H_tr, H_va = H.iloc[tr].values, H.iloc[va].values
        y_tr, y_va = y_struct[tr],    y_struct[va]
        
        # ——— fit pipeline ——————————————————————————————————————
        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("xgb", XGBRegressor(
                n_estimators=80,
                max_depth=4,
                learning_rate=0.3,
                objective="reg:squarederror",
                tree_method="hist"
            ))
        ])
        pipe.fit(X_tr, H_tr)
        
        # ——— score by IPCW‐C‑index on -H (higher hazard=risk) ———————
        pred_H = pipe.predict(X_va)
        ci = concordance_index_ipcw(y_tr, y_va, -pred_H)[0]
        print(f"Fold {fold:>2d} IPCW‑C index = {ci:.4f}")
        results["XGB_NA"].append(ci)
    
    mean_ci = np.mean(results["XGB_NA"])
    std_ci  = np.std(results["XGB_NA"])
    print(f"\nCV IPCW‑C index: {mean_ci:.4f} ± {std_ci:.4f}")
    
    # ——— refit on all data & save ————————————————————————————
    final_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("xgb", XGBRegressor(
            n_estimators=400,
            max_depth=4,
            learning_rate=0.05,
            objective="reg:squarederror",
            tree_method="hist"
        ))
    ])
    final_pipe.fit(X, H.values)
    joblib.dump(final_pipe, "best_survival_model.pkl")
    print("✅  Saved best_survival_model.pkl")

if __name__ == "__main__":
    main()


Fold  1 IPCW‑C index = 0.9018
Fold  2 IPCW‑C index = 0.8432
Fold  3 IPCW‑C index = 0.9353
Fold  4 IPCW‑C index = 0.7928
Fold  5 IPCW‑C index = 0.9317

CV IPCW‑C index: 0.8809 ± 0.0551
✅  Saved best_survival_model.pkl
